In [1]:
from facebook_business.adobjects.leadgenform import LeadgenForm
from facebook_business.api import FacebookAdsApi
import os
from dotenv import load_dotenv
import requests
import json

/Users/lroberts/bulkuploader/venv/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()

True

In [3]:
# Initialize the API
FacebookAdsApi.init(access_token=os.getenv('FACEBOOK_ACCESS_TOKEN'),
                    app_id=os.getenv('FACEBOOK_APP_ID'),
                    app_secret=os.getenv('FACEBOOK_APP_SECRET'))

# The ID of the existing lead form you want to copy
existing_lead_form_id = '1778956992538844'

In [6]:
def get_lead_form_data(lead_form_id):
    FacebookAdsApi.init(access_token=os.getenv('FACEBOOK_ACCESS_TOKEN'),
                        app_id=os.getenv('FACEBOOK_APP_ID'),
                        app_secret=os.getenv('FACEBOOK_APP_SECRET'))
    
    lead_form = LeadgenForm(lead_form_id)
    fields = [
        'context_card',
        'follow_up_action_url',
        'is_optimized_for_quality',
        'legal_content',
        'name',
        'page_id',
        'privacy_policy_url',
        'question_page_custom_headline',
        'questions',
        'thank_you_page'
    ]
    
    lead_form_data = lead_form.api_get(fields=fields)

    # Function to handle the export of data
    def export_or_use_directly(item):
        return item.export_all_data() if hasattr(item, 'export_all_data') else item

    # Organizing the data into a dictionary
    structured_data = {
        'name': lead_form_data.get('name'),
        'follow_up_action_url': lead_form_data.get('follow_up_action_url'),
        'is_optimized_for_quality': lead_form_data.get('is_optimized_for_quality'),
        'legal_content': export_or_use_directly(lead_form_data.get('legal_content')),
        'page_id': lead_form_data.get('page_id'),
        'privacy_policy_url': lead_form_data.get('privacy_policy_url'),
        'question_page_custom_headline': lead_form_data.get('question_page_custom_headline'),
        'questions': [export_or_use_directly(q) for q in lead_form_data.get('questions', [])],
        'context_card': export_or_use_directly(lead_form_data.get('context_card')),
        'thank_you_page': export_or_use_directly(lead_form_data.get('thank_you_page'))
    }

    return structured_data


In [10]:
structured_data = get_lead_form_data(existing_lead_form_id)

In [11]:
print(structured_data)

{'name': 'Dynamic Q4 2023', 'follow_up_action_url': 'https://www.orangetheory.com/en-us/locations/pennsylvania/coraopolis/630-chauvet-dr/', 'is_optimized_for_quality': False, 'legal_content': {'id': '852114545776940', 'privacy_policy': {'url': 'https://www.orangetheory.com/en-us/privacy-policy/'}, 'custom_disclaimer': {'title': 'Terms and Conditions', 'body': {'text': 'By submitting your information, you agree to the terms and conditions stated in our privacy policy.'}, 'checkboxes': [{'id': '1306593883549960', 'is_required': False, 'is_checked_by_default': False, 'text': 'Yes, send me transaction and reminder emails/texts.', 'key': 'yes,_send_me_transaction_and_reminder_emails/texts.'}, {'id': '252096110938568', 'is_required': False, 'is_checked_by_default': False, 'text': 'Yes, send me marketing emails/texts.', 'key': 'yes,_send_me_marketing_emails/texts.__'}]}}, 'page_id': '441544299531815', 'privacy_policy_url': 'https://www.orangetheory.com/en-us/privacy-policy/', 'question_page_c

In [13]:
# Assuming structured_data is the data fetched from the original lead form
new_page_id = "877867295627373"  # Replace with the ID of the new page
new_follow_up_action_url = "https://www.orangetheory.com/en-us/locations/massachusetts/dedham/610-providence-highway"  # Replace with the new URL

# Update the page_id and URLs
structured_data['page_id'] = new_page_id
structured_data['follow_up_action_url'] = new_follow_up_action_url

# Remove 'id' key from each question
for question in structured_data.get('questions', []):
    if 'id' in question:
        question.pop('id')

# Serialize the data
json_data = json.dumps(structured_data)

# Setup API endpoint and headers
page_id = structured_data['page_id']  # Ensure this is the new page ID
api_endpoint = f'https://graph.facebook.com/v18.0/{page_id}/leadgen_forms'
headers = {
    'Authorization': f'Bearer {os.getenv("FACEBOOK_ACCESS_TOKEN")}',
    'Content-Type': 'application/json'
}

# Make the API call
response = requests.post(api_endpoint, data=json_data, headers=headers)

# Handle the response
if response.status_code == 200:
    print("New lead form created successfully.")
    print(response.json())  # This should contain details of the new lead form
else:
    print("Failed to create new lead form.")
    print(response.text)  # This should contain error information

Failed to create new lead form.
{"error":{"message":"(#100) Invalid keys \"id, cover_photo\" were found in param \"context_card\".","type":"OAuthException","code":100,"fbtrace_id":"AguleDC9tvjxie2SuSDcQtr"}}
